In [2]:
import pandas as pd
import numpy as np

### process different phenotype files

In [3]:
polypred_max1 = pd.read_csv("/gpfs/commons/home/tlin/output/bellenguez/bellenguez_all_2/polypred/PLINKupdate_max1.prs",sep='\t')
polypred_max3 = pd.read_csv("/gpfs/commons/home/tlin/output/bellenguez/bellenguez_all_2/polypred/PLINKupdate_max3.prs",sep='\t')
polypred_max5 = pd.read_csv("/gpfs/commons/home/tlin/output/bellenguez/bellenguez_all_2/polypred/PLINKupdate_max5.prs",sep='\t')
polypred_max7 = pd.read_csv("/gpfs/commons/home/tlin/output/bellenguez/bellenguez_all_2/polypred/PLINKupdate_max7.prs",sep='\t')
polypred_max10 = pd.read_csv("/gpfs/commons/home/tlin/output/bellenguez/bellenguez_all_2/polypred/PLINKupdate_max10.prs",sep='\t')
manifest = pd.read_csv("/gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/ADSP_vcf/compact_filtered_vcf_16906/phenotype_data_10_28_2021/SampleManifest_DS_2021.02.19.v2_ALL.txt", sep='\t') ## the one with mapping info
phenotype1 = pd.read_csv("/gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/ADSP_vcf/compact_filtered_vcf_16906/phenotype_data_10_28_2021/ADNIPhenotypes_DS_2021.02.19.v2_ALL.txt", sep='\t')
phenotype2 = pd.read_csv("/gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/ADSP_vcf/compact_filtered_vcf_16906/phenotype_data_10_28_2021/ADSPCaseControlPhenotypes_DS_2021.02.19.v2_ALL.txt", sep='\t')
phenotype3 = pd.read_csv("/gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/ADSP_vcf/compact_filtered_vcf_16906/phenotype_data_10_28_2021/ADSPFamilyBasedPhenotypes_DS_2021.02.19.v2_ALL.txt", sep='\t', encoding='cp1252')
phenotype4 = pd.read_csv("/gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/ADSP_vcf/compact_filtered_vcf_16906/phenotype_data_10_28_2021/ADSP_phenotype_filtered.tsv", sep='\t')

ADSP_duplicate = pd.read_excel("/gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/ADSP_vcf/compact_filtered_vcf_16906/phenotype_data_10_28_2021/gcad.r3.wgs.16905.2021.08.24.pairwise_IBD.summary.xlsx", sep= '\t')
ancestry = pd.read_csv("/gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/ADSP_vcf/compact_filtered_vcf_16906/phenotype_data/ADSP_ethnicity_prediction.tsv", sep = "\t")
#phenotype = pd.read_csv("/gpfs/commons/home/tlin/data/phenotype_0219.2021.tsv", sep = '\t')


In [4]:
ADSP_duplicate[ADSP_duplicate.SUBJID1 == 'A-MIA-UM000336']

,SUBJID1,SampleID1,Sample_Set1,ReplicateAnalysisID1,SUBJID2,SampleID2,Sample_Set2,ReplicateAnalysisID2,PI_HAT,Notes (suggested duplicate to drop for ADSP focused analyses)


In [18]:
drop = np.array(ADSP_duplicate[ADSP_duplicate.loc[:,"Notes (suggested duplicate to drop for ADSP focused analyses)"] != "Drop both"]["Notes (suggested duplicate to drop for ADSP focused analyses)"])
drop_both = ADSP_duplicate[ADSP_duplicate.loc[:,"Notes (suggested duplicate to drop for ADSP focused analyses)"] == "Drop both"].loc[:,["SampleID1","SampleID2"]]
drop_both = np.append(np.array(drop_both.SampleID1),np.array(drop_both.SampleID2))
drop_sampleID = np.append(drop_both,drop) ## 306: drop both, 287: suggested to drop

In [19]:
len(drop_sampleID)

593

In [22]:
print(drop_sampleID.shape)
print(np.unique(drop_sampleID).shape)
drop_sampleID = np.unique(drop_sampleID)  # there are only 362 unique values

(593,)
(362,)


In [124]:
drop_SUBJID = manifest[manifest.SampleID.isin(drop_sampleID)].SUBJID
drop_SAMPID = manifest[manifest.SampleID.isin(drop_sampleID)].SampleID

## exploring ancestry file.

In [24]:
print(ancestry.shape)
print(ancestry[ancestry.sample_id.isin(manifest.SampleID)].shape)  ## There is one sample id missing out from manifest file. (Can't find its SUBJID: A-ADC-AD005043-BR-NCR-09AD15795)
ancestry[~ancestry.sample_id.isin(manifest.SampleID)]

(16906, 18)
(16905, 18)


,sample_id,population_pred,max_probability,prob_0,prob_1,prob_2,prob_3,prob_4,distance,threshold,population_code,superpopulation_code,super_population_name,population_name,name_concat,race_description,ethnicity_description,final_population
1418,A-ADC-AD005043-BR-NCR-09AD15795,AFR,0.988527,0.988527,0.005296,0.002237,0.001738,0.002202,141.635678,Not Matched,PUR,AMR,American Ancestry,Puerto Rican,American Ancestry_Puerto Rican,White,Not Hispanic,AFR


In [130]:
ancestry = ancestry[~ancestry.sample_id.isin(drop_SAMPID)]

In [31]:
# print(mapping.columns)
# print(ancestry[ancestry.sample_id.isin(mapping.oldSampleID)].shape)
# print(mapping.shape)


Index(['oldSUBJID', 'oldSampleID', 'SUBJID', 'SampleID'], dtype='object')
(23, 18)
(61, 4)


In [131]:
## add SUBJID to ancestry
ancestry = ancestry.merge(manifest, left_on="sample_id", right_on="SampleID")
ancestry.shape  ## (16905, 28)

(16544, 28)

In [28]:
phenotype1 = phenotype1[phenotype1.SUBJID.isin(ancestry.SUBJID)]
phenotype2 = phenotype2[phenotype2.SUBJID.isin(ancestry.SUBJID)]
phenotype3 = phenotype3[phenotype3.SUBJID.isin(ancestry.SUBJID)]

In [29]:
phenotype1.shape

(1566, 30)

In [31]:
phenotype2[phenotype2.SUBJID.isin(phenotype3.SUBJID)].shape  ## there are overlap within family-based and case control based ADSP phenotypic file. 

(31, 27)

In [33]:
## there is no NAs in phenotype1
phenotype1["Diagnosis"] = phenotype1[['PrevAD', 'IncAD']].max(axis=1).astype(int)
phenotype1['Age'] = np.where(phenotype1.Diagnosis == 1,phenotype1.Age_AD_onset, phenotype1.Age_current)

phenotype2[["PrevAD",'IncAD']] = phenotype2[["PrevAD",'IncAD']].fillna(value = -1)
phenotype2["Diagnosis"] = phenotype2[['PrevAD', 'IncAD']].max(axis=1).astype(int)

np.where(phenotype3.AD.isnull())  # there is no NA in phenotype3
phenotype3["Diagnosis"] = np.where((phenotype3.AD<= 3) & (phenotype3.AD > 0), 1,0)

In [42]:
phenotype1["Source"] = "ADNI"
phenotype2["Source"] = "ADSPCaseControl"
phenotype3["Source"] = "ADSPFamilyBased"

In [43]:
wanted_col = ["SUBJID","Sex","Age","APOE","Race","Diagnosis","Source"]

def check_na(df,name):
    print("phenotye: ",name)
    print("data size: ", df.loc[:,wanted_col].shape[0])
    print("NAs in Diagnosis: ", df[df.Diagnosis == -1].shape[0])
    print("NAs in Age ", df.loc[:,wanted_col].shape[0] -df.loc[:,wanted_col].dropna(subset = ["Age"]).shape[0])
    print("NAs in APOE ", df.loc[:,wanted_col].shape[0] -df.loc[:,wanted_col].dropna(subset = ["APOE"]).shape[0])
    print("NAs in Race ", df.loc[:,wanted_col].shape[0] -df.loc[:,wanted_col].dropna(subset = ["Race"]).shape[0],"\n")

    
check_na(phenotype1,"ADNI Phenotypes")
check_na(phenotype2,"ADSP CaseControlPhenotypes")
check_na(phenotype3,"ADSP FamilyBasedPhenotypes")



phenotye:  ADNI Phenotypes
data size:  1566
NAs in Diagnosis:  0
NAs in Age  6
NAs in APOE  0
NAs in Race  1 

phenotye:  ADSP CaseControlPhenotypes
data size:  12365
NAs in Diagnosis:  737
NAs in Age  1266
NAs in APOE  137
NAs in Race  14 

phenotye:  ADSP FamilyBasedPhenotypes
data size:  917
NAs in Diagnosis:  0
NAs in Age  41
NAs in APOE  5
NAs in Race  3 



In [86]:
phenotype_merge = pd.concat([phenotype1.loc[:,wanted_col],phenotype2.loc[:,wanted_col],phenotype3.loc[:,wanted_col]])
print(phenotype_merge.shape[0])  # phenotype file not omitting Diagnosis  = unknown 
print(phenotype_merge[phenotype_merge.Diagnosis == -1].shape[0])  # unknown cases in the phenotype_merge file

phenotype_merge.loc[phenotype_merge["Age"] == "90+","Age"] = 90

14848
737


In [87]:
phenotype_merge = phenotype_merge[~phenotype_merge.SUBJID.isin(drop_SUBJID)]
phenotype_merge.shape  ## removes 222 rows

(14626, 7)

In [48]:
duplicated_SUBJID = phenotype_merge[phenotype_merge.SUBJID.duplicated()].sort_values("SUBJID").SUBJID

In [79]:
phenotype_merge[phenotype_merge.SUBJID.isin(duplicated_SUBJID)].sort_values("SUBJID") ## most of them are duplicates, except for "A-MIA-UM000402", "A-MIA-UM000759","A-MIA-UM009911" (take family based).  A-RAS-RA000011 -> take case control

,SUBJID,Sex,Age,APOE,Race,Diagnosis,Source
11122,A-CUHS-CU000169,1,87,33.0,5.0,0,ADSPCaseControl
2683,A-CUHS-CU000169,1,87,33.0,5.0,0,ADSPFamilyBased
11143,A-CUHS-CU003023,0,86,33.0,5.0,0,ADSPCaseControl
4616,A-CUHS-CU003023,0,86,33.0,5.0,0,ADSPFamilyBased
13272,A-LOAD-LD002055,0,80,33.0,5.0,1,ADSPCaseControl
6111,A-LOAD-LD002055,0,80,33.0,5.0,1,ADSPFamilyBased
14893,A-MIA-UM000329,0,73,34.0,5.0,1,ADSPCaseControl
10324,A-MIA-UM000329,0,73,34.0,5.0,1,ADSPFamilyBased
14894,A-MIA-UM000336,1,72,34.0,5.0,1,ADSPCaseControl
10329,A-MIA-UM000336,1,72,34.0,5.0,1,ADSPFamilyBased


In [81]:
remove = phenotype_merge[(phenotype_merge.SUBJID.isin(duplicated_SUBJID)) & (phenotype_merge.Source ==  "ADSPCaseControl") & (phenotype_merge.SUBJID != "A-VAN-VU000119")]
remove = remove.append(phenotype_merge[(phenotype_merge.SUBJID == "A-VAN-VU000119") & (phenotype_merge.Source ==  "ADSPFamilyBased")])

In [83]:
remove.shape

(20, 7)

In [92]:
phenotype_merge = pd.concat([phenotype_merge, remove]).drop_duplicates(keep=False)
phenotype_merge.shape

(14606, 7)

In [22]:
# phenotype_merge2 = pd.concat([phenotype1.loc[:,wanted_col],phenotype2.loc[:,wanted_col],phenotype3.loc[:,wanted_col]]) # to not be confused with the previous phenotype file

# print(phenotype.shape[0])  # original faulty phenotype file
# print(phenotype_merge2.shape[0])  # phenotype file not omitting Diagnosis  = unknown 
# print(phenotype_merge2[phenotype_merge2.Diagnosis == -1].shape[0])  # unknown cases in the phenotype_merge2 file

# phenotype_merge2.loc[phenotype_merge2["Age"] == "90+","Age"] = 90
# ancestry[ancestry.SUBJID.isin(phenotype_merge2.SUBJID)].shape

# phenotype_test = phenotype_merge2
# phenotype_merge2 = phenotype_merge2[phenotype_merge2.SUBJID.isin(ancestry.SUBJID)]
# print(phenotype_merge2.shape)
# print("number of duplicates: ",phenotype_merge2.shape[0]- phenotype_merge2.drop_duplicates().shape[0])
# phenotype_merge2 = phenotype_merge2.drop_duplicates()  
# print(phenotype_merge2.shape) ## 14822 


# phenotype_merge2 = phenotype_merge2.sort_values("SUBJID")
# phenotype_merge2.reset_index(drop=True, inplace=True)
# phenotype_merge2[phenotype_merge2.SUBJID.isin(phenotype_merge2[phenotype_merge2.SUBJID.duplicated()].SUBJID)]


# ## manually deal with the 4 duplicated SUBJID 
# phenotype_merge2 = phenotype_merge2.drop([8486,11231], axis=0)
# index = phenotype_merge2[(phenotype_merge2.SUBJID.isin(phenotype_merge2[phenotype_merge2.SUBJID.duplicated()].SUBJID)) & (phenotype_merge2.Diagnosis == -1)].index
# phenotype_merge2 = phenotype_merge2.drop(index, axis=0)


14911
41537
1384


(14913, 28)

(14848, 6)
number of duplicates:  26
(14822, 6)


In [42]:
phenotype_merge2[phenotype_merge2.SUBJID.duplicated()]

,SUBJID,Sex,Age,APOE,Race,Diagnosis
1423,A-MIA-UM000474,1,73,34.0,5.0,1
10387,A-MIA-UM000402,0,NaN,34.0,5.0,0
10416,A-MIA-UM000759,0,NaN,34.0,5.0,0
10663,A-MIA-UM009911,0,NaN,34.0,4.0,0
11290,A-RAS-RA000011,1,85,34.0,5.0,1


,SUBJID,Sex,Age,APOE,Race,Diagnosis
8482,A-MIA-UM000402,0,NaN,34.0,5.0,-1
8483,A-MIA-UM000402,0,NaN,34.0,5.0,0
8485,A-MIA-UM000474,1,73,34.0,5.0,1
8486,A-MIA-UM000474,1,7383,NaN,NaN,1
8498,A-MIA-UM000759,0,NaN,34.0,5.0,0
8499,A-MIA-UM000759,0,NaN,34.0,5.0,-1
9202,A-MIA-UM009911,0,NaN,34.0,4.0,0
9203,A-MIA-UM009911,0,NaN,34.0,4.0,-1
11230,A-RAS-RA000011,1,75,34.0,5.0,1
11231,A-RAS-RA000011,1,85,34.0,5.0,1


In [46]:
phenotype_merge2[phenotype_merge2.SUBJID.isin(phenotype_merge2[phenotype_merge2.SUBJID.duplicated()].SUBJID)]

,SUBJID,Sex,Age,APOE,Race,Diagnosis


In [47]:
ancestry_merge = ancestry.merge(phenotype_merge2, on = "SUBJID")
print(ancestry_merge.shape)
print(ancestry_merge[ancestry_merge.Diagnosis == -1].shape) ## in 14911 samples, there are 738 without AD labels

(14913, 33)
(736, 33)


In [50]:
ancestry[~ancestry.SUBJID.isin(phenotype_merge2.SUBJID)][~ancestry[~ancestry.SUBJID.isin(phenotype_merge2.SUBJID)].SUBJID.isin((ancestry[(~ancestry.SUBJID.isin(phenotype1.SUBJID)) & (~ancestry.SUBJID.isin(phenotype2.SUBJID)) & (~ancestry.SUBJID.isin(phenotype3.SUBJID))].SUBJID))].SUBJID

Series([], Name: SUBJID, dtype: object)

In [54]:
ancestry[~ancestry.SUBJID.isin(ancestry_merge.SUBJID)].shape ## there are ~2k missing

(1992, 28)

In [55]:
print(ancestry_merge.shape[0])
print(ancestry_merge.shape[0] +1993)

14913
16906


In [64]:
ancestry_merge[ancestry_merge.Diagnosis== -1].shape

(736, 33)

In [93]:
chirag_all= pd.read_csv("/gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/ADSP_vcf/compact_filtered_vcf_16906/phenotype_data_10_28_2021/all_phenotypes_unique_ancestry_all_all.tsv",sep = '\t')
chirag_subset = pd.read_csv("/gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/ADSP_vcf/compact_filtered_vcf_16906/phenotype_data_10_28_2021/all_phenotypes_unique_ancestry_subset.tsv", sep = '\t')

In [94]:
print(chirag_all.shape)
print(chirag_subset.shape)

(14668, 32)
(13265, 32)


In [98]:
notfound = chirag_all[~chirag_all.SUBJID.isin(phenotype_merge.SUBJID)].SUBJID ## 68

In [106]:
phenotype3[phenotype3.SUBJID.isin(notfound)].iloc[:,1:20]

,FamID,Father,Mother,Sex,AD,Age,Age_baseline,APOE,AUTOPSY,Braak,Race,Ethnicity,FamGrp,Comments,Duplicate_SUBJID,Latest_Update_Version,Base_AD,Base_Age,Base_AUTOPSY
332,CU0026F,A-CUHS-CU001161,A-CUHS-CU001162,1,2,89,85,23.0,NaN,NaN,6.0,1.0,2.0,NaN,0,ng00067.v5,2,89,NaN
335,CU0026F,A-CUHS-CU001164,A-CUHS-CU001163,1,2,63,67,34.0,NaN,NaN,6.0,1.0,2.0,NaN,0,ng00067.v5,2,63,NaN
336,CU0026F,A-CUHS-CU001164,A-CUHS-CU001163,1,0,70,65,33.0,NaN,NaN,6.0,1.0,2.0,NaN,0,ng00067.v5,0,70,NaN
954,LD0949F,A-LOAD-LD004369,A-LOAD-LD004368,1,0,53,53,34.0,NaN,NaN,5.0,0.0,1.0,NaN,0,ng00067.v5,0,53,NaN
959,LD0949F,A-LOAD-LD004374,A-LOAD-LD004373,0,2,65,71,33.0,NaN,NaN,5.0,0.0,1.0,NaN,0,ng00067.v5,2,65,NaN
1412,UM0152F,A-MIA-UM000465,A-MIA-UM000466,1,1,70,86,34.0,NaN,5.0,5.0,NaN,1.0,"ng00067.v5: AD: mild neuronal loss, LB present...",1,ng00067.v5,1,70,NaN
1423,UM0152F,A-MIA-UM000465,A-MIA-UM000466,1,1,73,83,34.0,NaN,4.0,5.0,NaN,1.0,ng00067.v5: AD: infarcts,0,ng00067.v5,1,73,NaN
1426,UM0152F,A-MIA-UM000465,A-MIA-UM000466,1,1,78,79,23.0,NaN,5.0,5.0,NaN,1.0,NaN,1,ng00067.v5,1,78,NaN
1484,UM0196F,A-MIA-UM000867,A-MIA-UM000868,1,1,68,70,44.0,NaN,6.0,5.0,0.0,1.0,"ng00067.v5: AD: Parkinson Definite, Braak 5",0,ng00067.v5,1,68,NaN
1486,UM0196F,A-MIA-UM000870,A-MIA-UM000871,1,1,70,78,34.0,NaN,5.0,5.0,NaN,1.0,NaN,0,ng00067.v5,1,70,NaN


In [108]:
phenotype1[phenotype1.SUBJID.isin(notfound)].iloc[:,1:20]

,Sex,PrevAD,IncAD,Age_current,Age_baseline,Age_MCI_onset,Age_AD_onset,APOE,AUTOPSY,Braak,Race,Ethnicity,AD_last_visit,MCI_last_visit,Duplicate_SUBJID,Comments,Latest_Update_Version,Base_PrevAD,Base_IncAD


In [112]:
phenotype2[phenotype2.SUBJID.isin(notfound)].SUBJID

1471      A-ADC-AD000151
1729      A-ADC-AD000492
2004      A-ADC-AD000992
2184      A-ADC-AD001355
2616      A-ADC-AD001990
2774      A-ADC-AD002196
3520      A-ADC-AD003270
3912      A-ADC-AD003885
4845      A-ADC-AD005553
5390      A-ADC-AD006458
5391      A-ADC-AD006460
5529      A-ADC-AD006719
5553      A-ADC-AD006759
8781      A-ADC-AD012708
8792      A-ADC-AD012737
8808      A-ADC-AD012807
8880      A-ADC-AD012990
9321      A-ADC-AD013518
9349      A-ADC-AD013546
9356      A-ADC-AD013554
9359      A-ADC-AD013557
9377      A-ADC-AD013575
9390      A-ADC-AD013589
9391      A-ADC-AD013590
9395      A-ADC-AD013594
9399      A-ADC-AD013598
9435      A-ADC-AD013635
9437      A-ADC-AD013637
9454      A-ADC-AD013654
9530      A-ADC-AD013738
9539      A-ADC-AD013747
9562      A-ADC-AD013770
9580      A-ADC-AD013790
9585      A-ADC-AD013795
9590      A-ADC-AD013801
9598      A-ADC-AD013809
9622      A-ADC-AD013834
9660      A-ADC-AD013873
9802      A-ADC-AD014018
11469    A-CUHS-CU009833


In [111]:
test = phenotype2[phenotype2.SUBJID.isin(notfound)]
test[~test.SUBJID.isin(drop_SUBJID)]

,SUBJID,Sex,PrevAD,IncAD,Age,Age_baseline,APOE,AUTOPSY,Braak,Race,...,Base_AUTOPSY,Base_Braak,Base_AD,Base_Version,Update_Baseline,Update_Latest,Update_ADstatus,Correction,Diagnosis,Source


In [114]:
ADSP_duplicate[ADSP_duplicate.SUBJID1 == "A-ADC-AD000151"]

,SUBJID1,SampleID1,Sample_Set1,ReplicateAnalysisID1,SUBJID2,SampleID2,Sample_Set2,ReplicateAnalysisID2,PI_HAT,Notes (suggested duplicate to drop for ADSP focused analyses)
203,A-ADC-AD000151,A-ADC-AD000151-BR-NCR-09AD14539,snd10020,REP6,A-ADC-AD000151,A-ADC-AD000151-BR-NCR-09AD14539g,snd10013,REP6,1.0,A-ADC-AD000151-BR-NCR-09AD14539g


In [116]:
polypred_max10[polypred_max10.IID == 'A-ADC-AD000151-BR-NCR-09AD14539g']

,FID,IID,PRS
10456,A-ADC-AD000151-BR-NCR-09AD14539g,A-ADC-AD000151-BR-NCR-09AD14539g,-0.0001


In [121]:
manifest 

,SUBJID,SampleID,BODY_SITE,ANALYTE_TYPE,Sequencing_Center,Sequencing_Platform,SAMPLE_USE,Technical_Replicate,Study_DSS,Sample_Set
0,A-ACT-AC000004,A-ACT-AC000004-BL-UPN-15872,Blood,DNA,WashU,Illumina HiSeq 2000,WES,0,sa000001,snd10000
1,A-ACT-AC000007,A-ACT-AC000007-BL-UPN-6888,Blood,DNA,WashU,Illumina HiSeq 2000,WES,0,sa000001,snd10000
2,A-ACT-AC000008,A-ACT-AC000008-BL-UPN-8307,Blood,DNA,WashU,Illumina HiSeq 2000,WES,0,sa000001,snd10000
3,A-ACT-AC000010,A-ACT-AC000010-BL-UWA-15286,Blood,DNA,WashU,Illumina HiSeq 2000,WES,0,sa000001,snd10000
4,A-ACT-AC000014,A-ACT-AC000014-BL-UPN-23967,Blood,DNA,WashU,Illumina HiSeq 2000,WES,0,sa000001,snd10000
5,A-ACT-AC000015,A-ACT-AC000015-BL-UPN-39345,Blood,DNA,WashU,Illumina HiSeq 2000,WES,0,sa000001,snd10000
6,A-ACT-AC000016,A-ACT-AC000016-BL-UPN-7273,Blood,DNA,WashU,Illumina HiSeq 2000,WES,0,sa000001,snd10000
7,A-ACT-AC000020,A-ACT-AC000020-BL-UWA-13783,Blood,DNA,WashU,Illumina HiSeq 2000,WES,0,sa000001,snd10000
8,A-ACT-AC000022,A-ACT-AC000022-BL-UWA-15061,Blood,DNA,WashU,Illumina HiSeq 2000,WES,0,sa000001,snd10000
9,A-ACT-AC000023,A-ACT-AC000023-BL-UPN-5880,Blood,DNA,WashU,Illumina HiSeq 2000,WES,0,sa000001,snd10000


In [101]:
previous_pheno.shape

(14914, 33)

In [102]:
previous_pheno[~previous_pheno.sample_id.isin(ancestry_merge.sample_id)]

,sample_id,population_pred,max_probability,prob_0,prob_1,prob_2,prob_3,prob_4,distance,threshold,...,Sequencing_Platform,SAMPLE_USE,Technical_Replicate,Study_DSS,Sample_Set,Sex,Age,APOE,Race,Diagnosis
1419,A-ADC-AD005043-BR-NCR-09AD15795,AFR,0.988527,0.988527,0.005296,0.002237,0.001738,0.002202,141.635678,Not Matched,...,PCRFree_HiSeqX,WGS,0,sa000001,snd10020,1,78.0,34.0,5.0,1


In [91]:
ancestry_merge.columns

Index(['sample_id', 'population_pred', 'max_probability', 'prob_0', 'prob_1',
       'prob_2', 'prob_3', 'prob_4', 'distance', 'threshold',
       'population_code', 'superpopulation_code', 'super_population_name',
       'population_name', 'name_concat', 'race_description',
       'ethnicity_description', 'final_population', 'SUBJID', 'SampleID',
       'BODY_SITE', 'ANALYTE_TYPE', 'Sequencing_Center', 'Sequencing_Platform',
       'SAMPLE_USE', 'Technical_Replicate', 'Study_DSS', 'Sample_Set', 'Sex',
       'Age', 'APOE', 'Race', 'Diagnosis'],
      dtype='object')

In [92]:
previous_pheno.columns

Index(['sample_id', 'population_pred', 'max_probability', 'prob_0', 'prob_1',
       'prob_2', 'prob_3', 'prob_4', 'distance', 'threshold',
       'population_code', 'superpopulation_code', 'super_population_name',
       'population_name', 'name_concat', 'race_description',
       'ethnicity_description', 'final_population', 'SUBJID', 'SampleID',
       'BODY_SITE', 'ANALYTE_TYPE', 'Sequencing_Center', 'Sequencing_Platform',
       'SAMPLE_USE', 'Technical_Replicate', 'Study_DSS', 'Sample_Set', 'Sex',
       'Age', 'APOE', 'Race', 'Diagnosis'],
      dtype='object')

In [38]:
missing_in_merging = ancestry[~ancestry.SUBJID.isin(ancestry_merge.SUBJID)].loc[:,["SUBJID","SampleID"]]  #1995
missing_in_merging.to_csv("/gpfs/commons/home/tlin/data/missingID.tsv", sep = '\t', index = False)

In [103]:
previous_pheno[previous_pheno.sample_id == "A-ADC-AD005043-BR-NCR-09AD15795"]

,sample_id,population_pred,max_probability,prob_0,prob_1,prob_2,prob_3,prob_4,distance,threshold,...,Sequencing_Platform,SAMPLE_USE,Technical_Replicate,Study_DSS,Sample_Set,Sex,Age,APOE,Race,Diagnosis
1419,A-ADC-AD005043-BR-NCR-09AD15795,AFR,0.988527,0.988527,0.005296,0.002237,0.001738,0.002202,141.635678,Not Matched,...,PCRFree_HiSeqX,WGS,0,sa000001,snd10020,1,78.0,34.0,5.0,1


In [104]:
previous_pheno = previous_pheno.drop(1419, axis=0)

In [112]:
previous_pheno.eq(ancestry_merge)

,sample_id,population_pred,max_probability,prob_0,prob_1,prob_2,prob_3,prob_4,distance,threshold,...,Sequencing_Platform,SAMPLE_USE,Technical_Replicate,Study_DSS,Sample_Set,Sex,Age,APOE,Race,Diagnosis
0,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,False,True,True,True
1,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,False,True,True,True
2,True,True,True,False,True,False,True,True,True,True,...,True,True,True,True,True,True,False,True,True,True
3,True,True,True,False,True,False,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
4,True,True,True,False,True,True,True,True,True,True,...,True,True,True,True,True,True,False,True,True,True
5,True,True,True,True,False,True,True,True,False,True,...,True,True,True,True,True,True,False,True,True,True
6,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,False,True,True,True
7,True,True,False,True,True,True,False,True,True,True,...,True,True,True,True,True,True,False,True,True,True
8,True,True,True,True,False,True,True,True,True,True,...,True,True,True,True,True,True,False,True,True,True
9,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,False,True,True,True


In [106]:
previous_pheno.shape

(14913, 33)

In [107]:
ancestry_merge.shape

(14913, 33)

In [120]:
ancestry_merge = ancestry_merge.sort_values("sample_id")
previous_pheno = previous_pheno.sort_values("sample_id")

In [125]:
print(ancestry_merge.iloc[14884,:])
print(previous_pheno.iloc[14884,:])

sample_id                G-MSBB-MB000312-BR-MSBB-71973
population_pred                                    EUR
max_probability                               0.987433
prob_0                                      0.00525002
prob_1                                      0.00398193
prob_2                                      0.00168014
prob_3                                        0.987433
prob_4                                      0.00165519
distance                                        14.994
threshold                                      Matched
population_code                                    CEU
superpopulation_code                               EUR
super_population_name                European Ancestry
population_name                                   CEPH
name_concat                     European Ancestry_CEPH
race_description                                 White
ethnicity_description                     Not Hispanic
final_population                                   EUR
SUBJID    

In [124]:
ancestry_merge.eq(previous_pheno).iloc[14884,:]

sample_id                False
population_pred           True
max_probability           True
prob_0                    True
prob_1                    True
prob_2                    True
prob_3                    True
prob_4                    True
distance                 False
threshold                 True
population_code           True
superpopulation_code      True
super_population_name     True
population_name           True
name_concat               True
race_description          True
ethnicity_description     True
final_population          True
SUBJID                   False
SampleID                 False
BODY_SITE                 True
ANALYTE_TYPE              True
Sequencing_Center         True
Sequencing_Platform       True
SAMPLE_USE                True
Technical_Replicate       True
Study_DSS                 True
Sample_Set                True
Sex                       True
Age                      False
APOE                      True
Race                      True
Diagnosi

In [131]:
ancestry_merge.columns

Index(['sample_id', 'population_pred', 'max_probability', 'prob_0', 'prob_1',
       'prob_2', 'prob_3', 'prob_4', 'distance', 'threshold',
       'population_code', 'superpopulation_code', 'super_population_name',
       'population_name', 'name_concat', 'race_description',
       'ethnicity_description', 'final_population', 'SUBJID', 'SampleID',
       'BODY_SITE', 'ANALYTE_TYPE', 'Sequencing_Center', 'Sequencing_Platform',
       'SAMPLE_USE', 'Technical_Replicate', 'Study_DSS', 'Sample_Set', 'Sex',
       'Age', 'APOE', 'Race', 'Diagnosis'],
      dtype='object')

In [128]:
#phenotype = ancestry_merge.loc[:,["SUBJID","SampleID","Sex","Age","APOE","Race","Diagnosis"]]
#ancestry_merge.to_csv("/gpfs/commons/home/tlin/data/phenotype_0219.2021_v2.tsv",sep = '\t', index = False)
previous_pheno.to_csv("/gpfs/commons/home/tlin/data/phenotype_0219.2021_v1.tsv",sep = '\t', index = False)

In [62]:
phenotype_test[phenotype_test.SUBJID.isin(missing_in_merging.SUBJID)]

,SUBJID,Sex,Age,APOE,Race,Diagnosis
8489,A-MIA-UM000402,0,NaN,34.0,5.0,0
8490,A-MIA-UM000402,0,NaN,34.0,5.0,-1
8508,A-MIA-UM000759,0,NaN,34.0,5.0,0
8509,A-MIA-UM000759,0,NaN,34.0,5.0,-1
9228,A-MIA-UM009911,0,NaN,34.0,4.0,0
9229,A-MIA-UM009911,0,NaN,34.0,4.0,-1


In [136]:
#previous_pheno.merge(ancestry_merge, on='sample_id', how='outer', indicator=True)

ancestry_merge2 = ancestry_merge.loc[:,["SUBJID","SampleID","Sex","Age","APOE","Race","Diagnosis"]]
previous_pheno2 = previous_pheno.loc[:,["SUBJID","SampleID","Sex","Age","APOE","Race","Diagnosis"]]

In [179]:
previous_pheno2.iloc[:5,:].merge(ancestry_merge2.iloc[:5,:], on='SampleID', how='outer', indicator=False)

,SUBJID_x,SampleID,Sex_x,Age_x,APOE_x,Race_x,Diagnosis_x,SUBJID_y,Sex_y,Age_y,APOE_y,Race_y,Diagnosis_y
0,A-ACT-AC000014,A-ACT-AC000014-BL-NCR-15AD78694,0,77.0,23.0,5.0,0,A-ACT-AC000014,0,77,23.0,5.0,0
1,A-ACT-AC000034,A-ACT-AC000034-BL-NCR-16AD84906,0,68.0,33.0,5.0,0,A-ACT-AC000034,0,68,33.0,5.0,0
2,A-ACT-AC000057,A-ACT-AC000057-BL-NCR-15AD78356,0,88.0,33.0,5.0,1,A-ACT-AC000057,0,88,33.0,5.0,1
3,A-ACT-AC000072,A-ACT-AC000072-BL-NCR-15AD77250,1,90.0,33.0,5.0,1,A-ACT-AC000072,1,90,33.0,5.0,1
4,A-ACT-AC000088,A-ACT-AC000088-BL-NCR-15AD76548,1,86.0,33.0,5.0,1,A-ACT-AC000088,1,86,33.0,5.0,1


In [200]:
pd.concat([previous_pheno2,ancestry_merge2]).sort_values("SUBJID")

,SUBJID,SampleID,Sex,Age,APOE,Race,Diagnosis
0,A-ACT-AC000014,A-ACT-AC000014-BL-NCR-15AD78694,0,77,23.0,5.0,0
0,A-ACT-AC000014,A-ACT-AC000014-BL-NCR-15AD78694,0,77,23.0,5.0,0
1,A-ACT-AC000034,A-ACT-AC000034-BL-NCR-16AD84906,0,68,33.0,5.0,0
1,A-ACT-AC000034,A-ACT-AC000034-BL-NCR-16AD84906,0,68,33.0,5.0,0
2,A-ACT-AC000057,A-ACT-AC000057-BL-NCR-15AD78356,0,88,33.0,5.0,1
2,A-ACT-AC000057,A-ACT-AC000057-BL-NCR-15AD78356,0,88,33.0,5.0,1
3,A-ACT-AC000072,A-ACT-AC000072-BL-NCR-15AD77250,1,90,33.0,5.0,1
3,A-ACT-AC000072,A-ACT-AC000072-BL-NCR-15AD77250,1,90,33.0,5.0,1
4,A-ACT-AC000088,A-ACT-AC000088-BL-NCR-15AD76548,1,86,33.0,5.0,1
4,A-ACT-AC000088,A-ACT-AC000088-BL-NCR-15AD76548,1,86,33.0,5.0,1


In [194]:
print(ancestry_merge2.dtypes)
print(previous_pheno2.dtypes)

SUBJID        object
SampleID      object
Sex            int64
Age           object
APOE         float64
Race         float64
Diagnosis      int64
dtype: object
SUBJID        object
SampleID      object
Sex            int64
Age          float64
APOE         float64
Race         float64
Diagnosis      int64
dtype: object


In [197]:
previous_pheno2.Age = previous_pheno2.Age.astype(object)

AttributeError: 'Series' object has no attribute 'value'

In [58]:
print(phenotype_test[phenotype_test.SUBJID.isin(error.SUBJID)].shape)
print(phenotype_test[phenotype_test.SUBJID == 'A-PRHS-PR001575'])
print(phenotype_test[phenotype_test.SUBJID == 'A-PRHS-PR001569'])
print(phenotype_test[phenotype_test.SUBJID == 'A-PRHS-PR001557'])


(2245, 6)
                SUBJID  Sex Age  APOE  Race  Diagnosis
11255  A-PRHS-PR001575    1  77   NaN   6.0          0
                SUBJID  Sex Age  APOE  Race  Diagnosis
11249  A-PRHS-PR001569    1  72  33.0   6.0          0
                SUBJID  Sex Age  APOE  Race  Diagnosis
11237  A-PRHS-PR001557    1  66   NaN   6.0          0


### combine polypred result with phenotypic files

In [40]:
def combine_phenotype_polypred(polypred, phenotype):
    polypred = polypred.merge(phenotype, left_on = "IID", right_on = "SampleID").drop(["IID"],axis=1)
    polypred = polypred.loc[:,["SUBJID","SampleID","FID","PRS","Diagnosis","Sex","Age","APOE","Race"]]
    return(polypred)

In [41]:
polypred_pheno_max1 = combine_phenotype_polypred(polypred_max1, ancestry_merge)
polypred_pheno_max3 = combine_phenotype_polypred(polypred_max3, ancestry_merge)
polypred_pheno_max5 = combine_phenotype_polypred(polypred_max5, ancestry_merge)
polypred_pheno_max7 = combine_phenotype_polypred(polypred_max7, ancestry_merge)
polypred_pheno_max10 = combine_phenotype_polypred(polypred_max10, ancestry_merge)

In [42]:
print("check match/unmatch:")
print("max_SNP = 1, before merging:%s, after merging:%s"%(polypred_max1.shape[0],polypred_pheno_max1.shape[0]) )
print("max_SNP = 3, before merging:%s, after merging:%s"%(polypred_max3.shape[0],polypred_pheno_max3.shape[0]) )
print("max_SNP = 5, before merging:%s, after merging:%s"%(polypred_max5.shape[0],polypred_pheno_max5.shape[0]) )
print("max_SNP = 7, before merging:%s, after merging:%s"%(polypred_max7.shape[0],polypred_pheno_max7.shape[0]) )
print("max_SNP = 10, before merging:%s, after merging:%s"%(polypred_max10.shape[0],polypred_pheno_max10.shape[0]) )


check match/unmatch:
max_SNP = 1, before merging:16906, after merging:14914
max_SNP = 3, before merging:16906, after merging:14914
max_SNP = 5, before merging:16906, after merging:14914
max_SNP = 7, before merging:16906, after merging:14914
max_SNP = 10, before merging:16906, after merging:14914


In [43]:
polypred_pheno_max1.to_csv("/gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/ADSP_vcf/compact_filtered_vcf_16906/phenotype_data/diagnosis/prs_diagnosis_0219.2021_max_snp_1.tsv",index=False, sep = '\t')
polypred_pheno_max3.to_csv("/gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/ADSP_vcf/compact_filtered_vcf_16906/phenotype_data/diagnosis/prs_diagnosis_0219.2021_max_snp_3.tsv",index=False, sep = '\t')
polypred_pheno_max5.to_csv("/gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/ADSP_vcf/compact_filtered_vcf_16906/phenotype_data/diagnosis/prs_diagnosis_0219.2021_max_snp_5.tsv",index=False, sep = '\t')
polypred_pheno_max7.to_csv("/gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/ADSP_vcf/compact_filtered_vcf_16906/phenotype_data/diagnosis/prs_diagnosis_0219.2021_max_snp_7.tsv",index=False, sep = '\t')
polypred_pheno_max10.to_csv("/gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/ADSP_vcf/compact_filtered_vcf_16906/phenotype_data/diagnosis/prs_diagnosis_0219.2021_max_snp_10.tsv",index=False, sep = '\t') 

In [44]:
polypred_pheno_max1.to_csv("/gpfs/commons/home/tlin/data/prs/bellenguez_all_2/bellenguez_all2_prs_max_snp_1.tsv",index=False, sep = '\t')
polypred_pheno_max3.to_csv("/gpfs/commons/home/tlin/data/prs/bellenguez_all_2/bellenguez_all2_prs_diagnosis_0219.2021_max_snp_3.tsv",index=False, sep = '\t')
polypred_pheno_max5.to_csv("/gpfs/commons/home/tlin/data/prs/bellenguez_all_2/bellenguez_all2_prs_diagnosis_0219.2021_max_snp_5.tsv",index=False, sep = '\t')
polypred_pheno_max7.to_csv("/gpfs/commons/home/tlin/data/prs/bellenguez_all_2/bellenguez_all2_prs_diagnosis_0219.2021_max_snp_7.tsv",index=False, sep = '\t')
polypred_pheno_max10.to_csv("/gpfs/commons/home/tlin/data/prs/bellenguez_all_2/bellenguez_all2_prs_diagnosis_0219.2021_max_snp_10.tsv",index=False, sep = '\t') 

In [72]:
def find_missing(polypred, polypred_pheno):
    missing = polypred[~polypred.IID.isin(polypred_pheno.SampleID)]
    return(missing)

In [44]:
find_missing(polypred_max1, polypred_pheno_max1).to_csv("/gpfs/commons/home/tlin/data/missingID_max1.tsv", sep = '\t')
find_missing(polypred_max3, polypred_pheno_max3).to_csv("/gpfs/commons/home/tlin/data/missingID_max3.tsv", sep = '\t')
find_missing(polypred_max5, polypred_pheno_max5).to_csv("/gpfs/commons/home/tlin/data/missingID_max5.tsv", sep = '\t')
find_missing(polypred_max7, polypred_pheno_max7).to_csv("/gpfs/commons/home/tlin/data/missingID_max7.tsv", sep = '\t')
find_missing(polypred_max10, polypred_pheno_max10).to_csv("/gpfs/commons/home/tlin/data/missingID_max10.tsv", sep = '\t')

In [74]:
find_missing(polypred_max1, polypred_pheno_max1).shape

(1995, 3)